In [18]:
from torchvision import transforms as transforms
import torchvision.datasets as dset
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

import torch
import time

def load_dataset(args):
    transform = transforms.Compose([ # Transforming and augmenting images
        transforms.RandomHorizontalFlip(), # 랜덤 좌우 반전
        transforms.RandomVerticalFlip(), # 랜덤 상하 반전
        transforms.ColorJitter(), # 랜덤 색상필터
        transforms.RandomResizedCrop((args.imgsz, args.imgsz)), # 랜덤으로 리사이즈 후, cropping
        transforms.ToTensor(), # Tensor로 변환
        transforms.Normalize((0.485, 0.465, 0.406), (0.229, 0.224, 0.225))
    ])
    
    # 데이터셋 읽기
    train_data = dset.ImageNet('../../../../media/data1/data/Imagenet', split='train', transform=transform)
    test_data = dset.ImageNet('../../../../media/data1/data/Imagenet', split='val', transform=transform)
    
    return train_data, test_data

def pre_dset(rank, world_size, args):
    assert not args.batch_size % world_size, "batch size를 world size로 나뉘어져야 함, batch_size: " + str(args.batch_size) + ", world_size: " + str(world_size) + ", return value: " + str(args.batch_size % world_size)
    batch_per_gpu = args.batch_size // world_size # batch_size가 world_size의 몇 배인지(몫을 리턴)
    if rank == 0:
        print(f"{batch_per_gpu} batches per GPU...") # GPU 당 batch size
        
        
    train_set, test_set = load_dataset(args)
    
    
    # DDP로 분산 처리하기 위한 데이터 할당
    train_sampler = DistributedSampler(train_set, # 데이터셋
                                       num_replicas=world_size, # process 개수
                                       rank=rank, # process ID
                                       shuffle=args.shuffle, # shuffle 여부(DDP는 True, 단일은 False)
                                      )
    test_sampler = DistributedSampler(test_set,
                                      num_replicas=world_size, # process 개수
                                      rank=rank, # process ID
                                      shuffle=False, # Test set은 섞을 필요 없음
                                     )
    
    # DataLoader
    train_loader = DataLoader(train_set, # 데이터 셋
                              batch_size=batch_per_gpu, # GPU에 맞는 batch size(CPU는 그냥 batch_size)
                              pin_memory=args.pin_memory, # CPU에서 GPU의 VRAM으로 데이터를 로드해주기 위한 CPU의 Pinned memory
                              num_workers=args.num_workers, # 데이터 프로세싱 시 CPU 코어 할당량
                              sampler=train_sampler,
                              shuffle=not args.shuffle,  # shuffle옵션의 디폴트는 False이지만 여기서는 유동적으로 변경하였음
                             )
    test_loader = DataLoader(test_set,
                             batch_size=batch_per_gpu,
                             pin_memory=args.pin_memory, # CPU에서 GPU의 VRAM으로 데이터를 로드해주기 위한 CPU의 Pinned memory
                             num_workers=args.num_workers, # 데이터 프로세싱 시 CPU 코어 할당량
                             sampler=test_sampler,
                             shuffle=False
                            )

    return train_loader, test_loader

def train(model, train_loader, criterion, optimizer, rank, args) -> (float, float):
    model.train() # 학습을 위한 train mode로 변경
    running_loss, logging_loss, train_acc = 0, 0, 0  # 학습 loss, 로그 출력용 loss, 학습 Accuracy
    for epoch, (data, label) in enumerate(train_loader, 1):
        if args.pin_memory: # 고정된 메모리
            data, label = data.to(rank, non_blocking=True), label.to(rank, non_blocking=True) # non_blocking: 비동기로 GPU에 객체를 전달
        else:
            data, label = datda.to(rank), label.to(rank) # process Id만 지정
    
        out = model(data) # 모델에 데이터 넣기
        loss = criterion(out, label) # loss 함수로 입력과 라벨의 loss 구하기

        running_loss += loss.item()
        logging_loss += loss.item()
        if rank == 0:
            if not epoch % args.every: # 정해진 수마다 출력 (나눈 값의 나머지가 0이면 출력됨)
                print(f"[{args.epoch}/{len(train_loader)}] loss: {logging_loss / args.every}")
                logging_loss = 0

        train_acc += (out.detach().argmax(-1) == label).float().sum() / args.batch_size # accuracy

        optimizer.zero_grad() # 학습마다 기울기를 0으로 초기화
        loss.backward() # 가중치와 편향을 계산
        optimizer.step() # 가중치 업데이트

        return running_loss / len(train_loader), train_acc /len(train_loader) # loss 및 acc 리턴

def validate(model, val_loader, criterion, rank, args):
    model.eval() # 검증 시에는 Batch Normalization과 Dropout를 학습과 다르게 설정해야 함
    val_loss, val_acc = 0, 0
    with torch.no_grad():
        for data, label in val_loader:
            data, label = data.to(rank), label.to(rank)
            
            out = model(data) # 모델에 데이터 넣기
            loss = criterion(out, label) # loss 함수로 입력과 라벨의 loss 구하기
            
            val_acc += (out.detach().argmax(-1) == label).float().sum() / args.batch_size # val accuracy
            val_loss += loss # val loss
            
        return val_acc / len(val_loader), val_loss / len(val_loader) # val loss 및 val acc 리턴
    
def save_ckpt(state, file_name="./model_checkpoint/best_model.pth") -> None:
    torch.save(state, file_name)

In [16]:
import os
import argparse

import torch
import torch.nn as nn
import torch.distributed as dist

from sched import scheduler
from torch.nn.parallel import DistributedDataParallel as DDP

from ResNet import ResNet, Bottleneck
from prepare_dataset import pre_dset
from train import train, validate
from utils import save_ckpt

def parse_args():
    parser = argparse.ArgumentParser(description="Imagenet Training")
    
    ## Config
    parser.add_argument("--exp", type=str, default="./model_checkpoint") # checkpoint를 저장할 경로
    
    ## Training
    parser.add_argument("--lr", type=float, default=1e-4) # 학습률
    parser.add_argument("--epochs", type=int, default=100) # 학습 횟수
    parser.add_argument("--batch_size", type=int, default=32) # batch size
    parser.add_argument("--every", type=int, default=100) # 학습 중간에 loss를 출력할 빈도 수
    parser.add_argument("--step_size", type=int, default=30) # StepLR의 step_size
    parser.add_argument("--gamma", type=float, default=0.1) # StepLR의 gamma
    
    ## Data loader
    parser.add_argument("--pin_memory", action='store_true') # 데이터를 CPU -> GPU로 옮길 때 사용할 memory(store_true: True를 저장)
    parser.add_argument("--num_workers", type=int, default=2) # DataLoder에서 사용할 CPU 코어 개수
    parser.add_argument("--shuffle", action='store_true') # DDP의 DistributedSampler에서 shuffle의 여부이며, Data Loader는 이와 반대로 지정(store_true: True를 저장)
    parser.add_argument("--imgsz", type=int, default=600) # RandomResizedCrop 시 crop output의 image size 지정

    parser.add_argument("--local_rank", type=int, default=0) # GPU process ID
    return parser.parse_args() # 입력받은 argument 리턴

def cleanup():
    dist.destroy_process_group() # 분산 학습 완료 후, 프로세스 초기화
    
def main(rank, world_size, args):
    torch.cuda.set_device(rank) # GPU의 각 프로세스 세팅
    
    train_loader, val_loader = pre_dset(rank, world_size, args) # 데이터 불러오기 및 전처리
    
    model = ResNet(Bottleneck, [3,4,6,3]).to(rank) # model 생성자
    model = DDP(model, device_ids=[rank], output_device=rank) # 병렬 처리를 위해 DDP에 model, process id를 넘겨줌
    
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr) # 최적화기법 및 learning rate 설정
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=args.step_size, gamma=args.gamma) # learning rate를 step_size마다 gamma를 곱하여 감소시킴
    criterion = nn.CrossEntropyLoss()
    
    if rank == 0: print(f"Start Imagenet Training")
    best_acc, best_loss = 0., float("inf") # Best value 초기화 시 acc는 가장 낮은 값, loss는 무한으로 설정
    
    for epoch in range(args.epochs):
        train_loader.sampler.set_epoch(epoch) # epoch마다 DistributedSampler에게 현재 epoch을 계속 전달해야 함
        train_loss, train_acc = train(model, train_loader, criterion, optimizer, rank, args) # Train
    
        val_acc, val_loss = validate(model, val_loader, criterion, rank, args)
        
        ## reason of using ones_like: 
        ## the container's value should be on the same device with the value it will contain
        g_acc = [torch.ones_like(val_acc) for _ in range(world_size)]
        g_loss = [torch.ones_like(val_loss) for _ in range(world_size)]

        # DistributedDataParallel.all_gather: 모든 프로세스의 tensor 를 모든 프로세스의 tensor_list 에 복사
        dist.all_gather(g_acc, val_acc)
        dist.all_gather(g_loss, val_loss)

        if rank == 0: # 첫 번째 GPU process
            val_acc = torch.stack(g_acc, dim=0) # torch.stack: 새로운 차원에 Tensor를 붙임
            val_loss = torch.stack(g_loss, dim=0)
            val_acc, val_loss = val_acc.mean(), val_loss.mean()
            print(f"EPOCH {epoch} VALID: acc = {val_acc}, loss = {val_loss}") # EPOCH 당 accuracy 및 loss 출력
            if val_acc > best_acc: # best accuracy 찾기
                save_ckpt ({
                    "epoch": epoch+1, # 왜 1을 더해
                    "state_dict": model.module.state_dict(), # 학습된 모델 저장
                    "optimizer": optimizer.state_dict(), # optimizer 저장
                    "scheduler": scheduler.state_dict(), # scheduler 저장
                }, file_name=os.path.join(args.exp, f"best_acc.pth")) # best loss 저장
            if val_loss > best_loss: # best loss 찾기
                save_ckpt({
                    "epoch": epoch+1,
                    "state_dict": model.module.state_dict(),
                    "optimizer": optimizer.state_dict(),
                    "scheduler": scheduler.state_dict(),
                }, file_name=os.path.join(args.exp, f"best_loss.path"))
            save_ckpt({
                "epoch": epoch+1,
                "state_dict": model.module.state_dict(),
                "optimizer": optimizer.state_dict(),
                "scheduler": scheduler.state_dict(),
            }, file_name=os.path.join(args.exp, f"last.pth"))
        scheduler.step() # scheduler 업데이트
        # we do not need it when training, since DDP automatically does it for us (in loss.backward());
        # we do not need it when gathering data, since dist.all_gather_object does it for us;
        # we need it when enforcing execution order of codes, say one process loads the model that another process saves (I can hardly imagine this scenario is needed).
        dist.barrier() # 모든 프로세스가 동기화되도록 맞춰줌
    
    cleanup()
    
if __name__ == "__main__":
    args = parse_args() # argument 받기
    args.local_rank = int(os.environ['LOCAL_RANK']) # torchrun 사용을 위한 Local Rank 전달
    print(args)
    
    dist.init_process_group("nccl")
    
    if "./model_checkpoint" not in args.exp: # 입력 받은 경로에 해당 폴더가 없으면
        args.exp = os.path.join("./model_checkpoint", args.exp) # 경로를 이어붙임
    os.makedirs(args.exp, exist_ok=True) # exist_ok=True: 폴더가 없으면 자동 생성
    
    main(rank=args.local_rank, world_size=dist.get_world_size(), args=args) # main에 GPU의 process ID, process 수 및 args를 넘겨줌

usage: ipykernel_launcher.py [-h] [--exp EXP] [--lr LR] [--epochs EPOCHS]
                             [--batch_size BATCH_SIZE] [--every EVERY]
                             [--step_size STEP_SIZE] [--gamma GAMMA]
                             [--pin_memory] [--num_workers NUM_WORKERS]
                             [--shuffle] [--imgsz IMGSZ]
                             [--local_rank LOCAL_RANK]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/kangjun/.local/share/jupyter/runtime/kernel-ef463698-9fbe-4c6e-b8bd-6ff159b6ecdb.json


SystemExit: 2

/home/kangjun/anaconda3/envs/torch0126/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
